In [ ]:
import torch, random, shutil, os, numpy as np, torch.nn as nn, time, random, shutil
import matplotlib.pyplot as plt
from scipy import integrate
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import r2_score
from tqdm import trange, tqdm


class CustomDataset(Dataset):
    """Custom dataset for loading images.
    Attributes: data: dataset (dtype: torch.Tensor)"""

    def __init__(self, dataset):
        """Initialize the dataset.
        Args: dataset: dataset to load (dtype: np.array)"""
        self.data = torch.Tensor(dataset)

    def __len__(self):
        """Return the length of the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """Get the item at the index idx.
        Args: idx: index of the item to get (dtype: int)
        Returns: x: item at the index idx (dtype: torch.Tensor)"""
        x = self.data[idx]
        return x

#### Different schedules --- variance preserving and variance exploding

Time evolving probability density follows the following PDE:

![alt text](./figures/pde.png "Title")

![alt text](./figures/schedules.png "Title")

In [ ]:
class VP:
    def __init__(self, beta_max, beta_min):
        self.beta_min = beta_min
        self.beta_max = beta_max
    def _beta_t(self, t):
        return self.beta_min + t*(self.beta_max - self.beta_min)
    def _alpha_t(self, t):
        return t*self.beta_min + 0.5 * t**2 * (self.beta_max - self.beta_min)
    def _drift(self, x, t):
        return -0.5*self._beta_t(t[:,None])*x
    def _marginal_prob_mean(self, t):
        return torch.exp(-0.5 * self._alpha_t(t))
    def _marginal_prob_std(self, t):
        return torch.sqrt(1 - torch.exp(-self._alpha_t(t)))
    def _diffusion_coeff(self, t):
        return torch.sqrt(self._beta_t(t))

class VE:
    def __init__(self, sigma_max, sigma_min):
        self.sigma = sigma_max
    def _drift(self, x, t):
        return torch.zeros(x.shape)
    def _marginal_prob_mean(self, t):
        return torch.ones((1,))
    def _marginal_prob_std(self, t):
        return torch.sqrt((self.sigma**(2 * t) - 1.) / 2. / np.log(self.sigma))
    def _diffusion_coeff(self, t):
        return self.sigma**t

## Score network

![alt text](./figures/loss.png "Title")

In [ ]:
class score_edm(torch.nn.Module):
    def __init__(self, device, x_dim=1, y_dim=1 ,width=256, depth=2, activation=nn.ReLU):
        super().__init__()

        self.width = width
        self.depth = depth
        self.activation = activation()
        self.dim = x_dim+y_dim

        net = []
        net.append(nn.Linear(self.dim+4,self.width))
        net.append(self.activation)
        for _ in range(self.depth):
            net.append(nn.Linear(self.width,self.width))
            net.append(self.activation)
        net.append(nn.Linear(self.width,x_dim))
        self.net = nn.Sequential(*net).to(device=device)

    def forward(self, x, y, t, mode=None):
        t = t.squeeze()
        embed = [t - 0.5, torch.cos(2*np.pi*t), torch.sin(2*np.pi*t), -torch.cos(4*np.pi*t)]
        embed = torch.stack(embed, dim=-1)

        x_in = torch.cat([x, y, embed], dim=-1)

        score = self.net(x_in).to(torch.float32)
        return score

def loss_func(net, X, y, schedule):
    t = torch.rand([X.shape[0], 1], device=X.device)
    noise = torch.randn_like(X)
    mean = (schedule._marginal_prob_mean(t)).to(X.device)
    std  = (schedule._marginal_prob_std(t)).to(X.device)
    x_tilde = mean * X + std * noise
    score = net(x_tilde, y, t)
    loss = torch.mean((std*score + noise)**2, dim=(1))
    return loss.mean()


## Training data

![alt text](./figures/problem.png "Title")

![alt text](./figures/problemdata.png "Title")

In [ ]:

N_samples = 20000
added_noise = 2
schedule_type = 'VP'
print(f"number of samples: {N_samples}")
print(f"added noise: {added_noise} %")

data = torch.from_numpy(np.load(f"adv_diff_dataset_with_noise_{added_noise}_normalized.npy")).float()
dim = data.shape[1]
n_segment = dim//2

N_train_samples = int(0.95*N_samples)
train_data = data[:N_train_samples]
test_data = data[N_train_samples:N_train_samples+10]
b_size = 1000
b_size_test = 2
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)
train_dataloader = DataLoader(train_dataset, batch_size=b_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=b_size_test, shuffle=False)

print(f"train data shape: {train_data.shape}")
print(f"test data shape: {test_data.shape}")


number of samples: 20000
added noise: 2 %
train data shape: torch.Size([19000, 60])
test data shape: torch.Size([10, 60])


## Training

In [ ]:
random.seed(1)
np.random.seed(1)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

n_epochs = 1000
lr = 1e-3
sigma_max = 15.0
sigma_min = 0.001
save_checkpoints_marks = [100,10000]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

width = 256
depth = 4
x_dim = n_segment
y_dim = n_segment

score_net = score_edm(device=device, x_dim=x_dim, y_dim=y_dim, width=width, depth=depth, activation=nn.ReLU)
optimizer = torch.optim.Adam(score_net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

if schedule_type==1:
    schedule = VE(sigma_max=sigma_max, sigma_min=sigma_min)
else:
    schedule = VP(sigma_max=sigma_max, sigma_min=sigma_min)

expname = 'samples' + str(N_samples) + 'noise_'+ str(added_noise) + \
          'depth_'+str(depth) + '_epochs_' + str(n_epochs) + \
          '_schedule_' + schedule.__class__.__name__ + '_sigma_' + str(int(sigma_max))

path_dir = f'./exps/'
path_to_run = path_dir + expname + '/'
if os.path.exists(path_to_run):
    shutil.rmtree(path_to_run)
os.makedirs(path_to_run)
os.makedirs(path_to_run+'checkpoints/')
os.makedirs(path_to_run+'figures/')

loss_list = []
pbar = tqdm(range(n_epochs), desc="Loss: ", ncols=100, colour='green')
for epoch in pbar:
    total_loss = 0.0
    for X in train_dataloader:
        score_net.train()
        X = X.to(device)  # increase batch size by repeating data
        x = X[:,0:x_dim]
        y = X[:,x_dim:]
        loss = loss_func(score_net, x, y, schedule)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    total_loss /= len(train_dataloader)
    loss_list.append(loss.item())
    if epoch % 50 == 0:
        pbar.set_description("Loss: {:.3f}".format(total_loss))

states = [score_net.state_dict(), score_net.state_dict(), optimizer.state_dict(), n_epochs]
torch.save(states, path_to_run+'checkpoints/'+'checkpoint.pth')

cuda


Loss: 0.032: 100%|██████████████████████████████████████████████| 1000/1000 [02:38<00:00,  6.32it/s]


In [ ]:
def euler_sampler(score_net, y, schedule, latents, batch_size, device, n_steps=2000, alpha=1.0):
    def score_eval_wrapper(sample, time_steps, schedule):
        with torch.no_grad():
            score = score_net(sample, y.to(device), time_steps)

        return score.detach().cpu()

    time_steps = torch.linspace(1.0, 0.0, n_steps)
    dt = time_steps[0] - time_steps[1]
    init_x = (latents * schedule._marginal_prob_std(time_steps[0]))
    x_path = torch.zeros(batch_size, latents.shape[1], n_steps)
    x_path[:,:,0] = init_x
    x = init_x

    with torch.no_grad():
        for (j,time) in enumerate(time_steps):
            batch_time = torch.ones(batch_size) * time
            f = schedule._drift(x, batch_time)
            g = schedule._diffusion_coeff(batch_time)
            drift = -1.*f + 0.5*(1+alpha)*(g**2)[:,None]*score_eval_wrapper(x.to(device), batch_time.to(device), schedule)
            x = x + dt * drift + torch.sqrt(alpha*dt)*g[:,None]*torch.randn_like(x)
            x_path[:,:,j] = x

    return time_steps.cpu().numpy(), x_path.cpu().numpy()


In [ ]:
def expand_tensor(tensor, n):
    expanded_tensor = tensor.repeat_interleave(n, dim=0)  # Efficiently repeat rows
    return expanded_tensor

generated_samples_list =[]

batch_size = 300

for X in test_dataloader:
    latents = torch.randn(batch_size*X.shape[0], x_dim)
    y = X[:,x_dim:]
    y_tensor = expand_tensor(y,batch_size)
    samples_t, samples_x = euler_sampler(score_net, y_tensor, schedule, latents, batch_size*X.shape[0], device)
    res_loc = torch.tensor(samples_x, device=latents.device, dtype=torch.float32).reshape(batch_size*X.shape[0], x_dim, len(samples_t))
    final_samples = res_loc[:,:,-1].detach().numpy()
    generated_samples_list.append(final_samples)

generated_samples = np.concatenate(generated_samples_list, axis=0)

print("size of the generated samples tensor:")
print(generated_samples.shape)


size of the generated samples tensor:
(3000, 30)


In [ ]:
real_samples = test_data[:,:x_dim].numpy()
x_mean = np.zeros([real_samples.shape[0],x_dim])
x_std = np.zeros([real_samples.shape[0],x_dim])

for i in range(real_samples.shape[0]):
    xg = generated_samples[batch_size*i:batch_size*(i+1)]
    x_mean[i,:] = np.mean(xg, axis=0)
    x_std[i,:] = np.std(xg, axis=0)

# np.savez(path_to_run + 'generated_mean_std.npz', array1=x_mean, array2=x_std, array3=real_samples)

real_mean = np.mean(np.mean(real_samples))
print(f"mean of the real data= {real_mean}")
mse_per_sample = np.mean(np.abs(real_samples - x_mean), axis=1)/real_mean
mse_per_segment = np.mean(np.abs(real_samples - x_mean), axis=0)/real_mean
ave_mse = np.mean(mse_per_sample)
print(f"average mean error = {ave_mse}")
r2score = r2_score(real_samples, x_mean)
print(f"coefficient of determination(r2 score) = {r2score}")


# counting in-bound (mean-std, mean+std) predictions
lower_bound = x_mean - x_std
upper_bound = x_mean + x_std

mean of the real data= 0.2887803614139557
average mean error = 0.1634379893612455
coefficient of determination(r2 score) = 0.7752524496720474


In [ ]:
# n_group_plots=random.sample(range(0, 1000), 100)
n_segment = real_samples.shape[1]//2

for i in range(test_data.shape[0]): #n_group_plots:
    # Create a new figure for each sample (1 row, 2 columns)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 14))  # 1 row, 2 columns

    # Upper wall flux (last 7 values)
    ax1.step(np.arange(n_segment+1), np.append(real_samples[i, n_segment], real_samples[i, n_segment:]), label="Real", color='black', linestyle='-', linewidth=1)
    ax1.step(np.arange(n_segment+1), np.append(x_mean[i, n_segment], x_mean[i, n_segment:]), label="Generated Mean", color='blue', linestyle='-', linewidth=2)
    ax1.step(np.arange(n_segment+1), np.append(upper_bound[i, n_segment] , upper_bound[i, n_segment:]), label="Generated Mean + Std", color='red', linestyle='--', linewidth=1)
    ax1.step(np.arange(n_segment+1), np.append(lower_bound[i, n_segment], lower_bound[i, n_segment:]), label="Generated Mean - Std", color='red', linestyle='--', linewidth=1)
    ax1.set_title(f"Sample {i+1} - Upper Wall Flux")
    ax1.set_xlabel("segment Index")
    ax1.set_ylabel("normalized Flux")
    ax1.grid(True)
    ax1.legend(loc="best")

    # Lower wall flux (first 7 values)
    ax2.step(np.arange(n_segment+1), np.append(real_samples[i, 0],real_samples[i, :n_segment]), label="Real", color='black', linestyle='-', linewidth=1)
    ax2.step(np.arange(n_segment+1), np.append(x_mean[i, 0], x_mean[i, :n_segment]), label="Generated Mean", color='blue', linestyle='-', linewidth=2)
    ax2.step(np.arange(n_segment+1), np.append(upper_bound[i, 0], upper_bound[i, :n_segment]), label="Generated Mean + Std", color='red', linestyle='--', linewidth=1)
    ax2.step(np.arange(n_segment+1), np.append(lower_bound[i, 0], lower_bound[i, :n_segment]), label="Generated Mean - Std", color='red', linestyle='--', linewidth=1)
    ax2.set_title(f"Sample {i+1} - Lower Wall Flux")
    ax2.set_xlabel("segment Index")
    ax2.set_ylabel("normalized Flux")
    ax2.grid(True)
    ax2.legend(loc="best")

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Save the figure as a PNG file
    plt.savefig(path_to_run + 'figures/' + f'sample_{i+1}_flux.png')

    # Close the figure to release memory
    plt.close()

![alt text](./figures/sample_1_flux.png "Title")